In [2]:
from quantum_viz import Viewer

# Create a quantum circuit that prepares a Bell state
circuit = {
    "qubits": [{ "id": 0 }, { "id": 1, "numChildren": 1 }],
    "operations": [
        {
            "gate": 'H',
            "targets": [{ "qId": 0 }],
        },
        {
            "gate": "X",
            "isControlled": "True",
            "controls": [{ "qId": 0 }],
            "targets": [{ "qId": 1 }],
        },
        {
            "gate": "Measure",
            "isMeasurement": "True",
            "controls": [{ "qId": 1 }],
            "targets": [{ "type": 1, "qId": 1, "cId": 0 }],
        },
    ],
}


widget = Viewer(circuit)
widget  # Display the widget

In [3]:
import qsharp

qsharp.init(target_profile=qsharp.TargetProfile.Base)

Q# initialized with configuration: {'targetProfile': 'base'}

In [4]:
%%qsharp

open Microsoft.Quantum.Measurement;
operation Foo() : Result[] {
    use q1 = Qubit();
    use q2 = Qubit();
    H(q1);
    CNOT(q1, q2);
    let rs = [MResetZ(q1),MResetZ(q2)];
    return []
}

In [5]:
%%qsharp

/// # Sample
/// Bell States
///
/// # Description
/// Bell states or EPR pairs are specific quantum states of two qubits
/// that represent the simplest (and maximal) examples of quantum entanglement.
///
/// This Q# program implements the four different Bell states.
namespace Sample {
    open Microsoft.Quantum.Diagnostics;
    open Microsoft.Quantum.Measurement;

    @EntryPoint()
    operation BellStates() : (Result, Result)[] {
        // Allocate the two qubits that will be used to create a Bell state.
        use register = Qubit[2];

        // This array contains a label and a preparation operation for each one
        // of the four Bell states.
        let bellStateTuples = [
            ("|Φ+〉", PreparePhiPlus),
            ("|Φ-〉", PreparePhiMinus),
            ("|Ψ+〉", PreparePsiPlus),
            ("|Ψ-〉", PreparePsiMinus)
        ];

        // Prepare all Bell states, show them using the `DumpMachine` operation
        // and measure the Bell state qubits.
        mutable measurements = [];
        for (label, prepare) in bellStateTuples {
            prepare(register);
            Message($"Bell state {label}:");
            DumpMachine();
            set measurements += [(MResetZ(register[0]), MResetZ(register[1]))];
        }
        return measurements;
    }

    operation PreparePhiPlus(register : Qubit[]) : Unit {
        ResetAll(register);             // |00〉
        H(register[0]);                 // |+0〉
        CNOT(register[0], register[1]); // 1/sqrt(2)(|00〉 + |11〉)
    }

    operation PreparePhiMinus(register : Qubit[]) : Unit {
        ResetAll(register);             // |00〉
        H(register[0]);                 // |+0〉
        Z(register[0]);                 // |-0〉
        CNOT(register[0], register[1]); // 1/sqrt(2)(|00〉 - |11〉)
    }

    operation PreparePsiPlus(register : Qubit[]) : Unit {
        ResetAll(register);             // |00〉
        H(register[0]);                 // |+0〉
        X(register[1]);                 // |+1〉
        CNOT(register[0], register[1]); // 1/sqrt(2)(|01〉 + |10〉)
    }

    operation PreparePsiMinus(register : Qubit[]) : Unit {
        ResetAll(register);             // |00〉
        H(register[0]);                 // |+0〉
        Z(register[0]);                 // |-0〉
        X(register[1]);                 // |-1〉
        CNOT(register[0], register[1]); // 1/sqrt(2)(|01〉 - |10〉)
    }
}


In [7]:
circuit = qsharp.circuit("Sample.BellStates()")
widget = Viewer(circuit)
widget